Optimize and plot the gains/VSWR of a logperiodic antenna (6 brass elements, 75 Ohm transmission lines) for both the 2.4GHz as the 5.8GHz ISM bands.
    Inspired by an excercise for a course, hence the weird constraints.


In [1]:
import numpy as np
import scipy.optimize
import matplotlib.pyplot as plt
import matplotlib as mpl

from PyNEC import *
from antenna_util import *

from context_clean import *

import math

brass_conductivity = 15600000 # mhos
copper_conductivity = 1.45e7 # Copper
ground_conductivity = 0.002
ground_dielectric = 10


In [2]:
def n_seg(freq, length):
  wavelength = 3e8/(1e6*freq)
  return (2 * (int(math.ceil(77*length/wavelength))/2)) + 1


def sc_quad_helix(height, diameter, wire_diameter = 0.02):
    
    nec = context_clean(nec_context())
    nec.set_extended_thin_wire_kernel(True)
    
    geo = geometry_clean(nec.get_geometry())

    wire_r = wire_diameter/2;
    helix_r = diameter/2;
    
    
    #print "Wire Diameter %s" % (wire_r * 2)
    
    helix_turns = 0.5
    
    # helix loop 
    helix_twist_height = height / helix_turns
    geo.helix(tag_id=1, nr_segments=50, spacing=helix_twist_height, lenght=height, start_radius=np.array([helix_r, 0]), end_radius=np.array([helix_r, 0]), wire_radius=wire_r)
    geo.move(rotate_z=90, copies=3, tag_inc=1)
    geo.wire(tag_id=10, nr_segments=2, src=np.array([0, 0, height]), dst=np.array([helix_r, 0, height]), radius=wire_r)
    geo.wire(tag_id=11, nr_segments=2, src=np.array([0, 0, height]), dst=np.array([0, helix_r, height]), radius=wire_r)
    geo.wire(tag_id=12, nr_segments=2, src=np.array([0, 0, height]), dst=np.array([-helix_r, 0, height]), radius=wire_r)
    geo.wire(tag_id=13, nr_segments=2, src=np.array([0, 0, height]), dst=np.array([0, -helix_r, height]), radius=wire_r)
    
    # Everything is copper
    nec.set_wire_conductivity(copper_conductivity)
    # finish structure definition
    nec.geometry_complete(ground_plane=False)

    # Voltage excitation at legs of the antenna
    nec.voltage_excitation(wire_tag=1, segment_nr=1, voltage=1.0)
    nec.voltage_excitation(wire_tag=2, segment_nr=1, voltage=0.0+1.0j)
    nec.voltage_excitation(wire_tag=3, segment_nr=1, voltage=-1.0)
    nec.voltage_excitation(wire_tag=4, segment_nr=1, voltage=0.0-1.0j)
    #nec.set_frequencies_linear(start_frequency=140, stop_frequency=150, count=100)
    #nec.radiation_pattern(thetas=Range(90, 90, count=1), phis=Range(180,180,count=1))

    return nec

In [3]:
start = 100
stop  = 150
count = stop - start

In [4]:
def get_gain_swr_range(height, diameter, start=start, stop=stop, step=1):
    gains_db = []
    frequencies = []
    vswrs = []
    for freq in range(start, stop + 1, step):
        nec = sc_quad_helix(height, diameter)
        nec.set_frequency(freq) # TODO: ensure that we don't need to re-generate this!
        nec.radiation_pattern(thetas=Range(90, 90, count=1), phis=Range(180,180,count=1))

        rp = nec.context.get_radiation_pattern(0)
        ipt = nec.get_input_parameters(0)
        z = ipt.get_impedance()[0]

        # Gains are in decibels
        gains_db.append(rp.get_gain()[0])
        vswrs.append(vswr(z, system_impedance))
        frequencies.append(ipt.get_frequency())

    return frequencies, gains_db, vswrs

def get_gain_swr(height, diameter, freq):
    nec = sc_quad_helix(height, diameter)
    nec.set_frequency(freq) # TODO: ensure that we don't need to re-generate this!
    nec.radiation_pattern(thetas=Range(90, 90, count=1), phis=Range(180,180,count=1))

    rp = nec.context.get_radiation_pattern(0)
    ipt = nec.get_input_parameters(0)
    z = ipt.get_impedance()[0]
    print z

    return ipt.get_frequency(), rp.get_gain()[0], vswr(z, system_impedance)

In [10]:
def create_optimization_target():
    def target(args):
        height, diameter  = args
        if height <= 0 or diameter <= 0:
            print "wrong element dimension"
            return float('inf')

        try:
            result = 0.0

            vswr_score = 0.0
            gains_score = 0.0

            freq, gain, vswr = get_gain_swr(height, diameter, freq=143.05)

            # VSWR should minimal in both bands, gains maximal:
            result = vswr - gain
            
        except Exception,e:
            print str(e)
            return float('inf')
        print result, height, diameter
        return result
    return target


In [6]:
def simulate_and_get_impedance(nec):
  nec.set_frequency(design_freq_mhz)

  nec.xq_card(0)

  index = 0
  return nec.get_input_parameters(index).get_impedance()[0]  # select only one impedance result (other are the same due to the structure symmetry)

system_impedance = 50 # This makes it a bit harder to optimize, given the 75 Ohm TLs, which is good for this excercise of course...

# (2.4 GHz to 2.5 GHz) and the 5.8 GHz ISM band (5.725 GHz to 5.875 GHz)

design_freq_mhz = 143 # The center of the first range
wavelength = 299792e3/(design_freq_mhz*1000000)

#majorLocator = mpl.ticker.MultipleLocator(10)
#majorFormatter = mpl.ticker.FormatStrFormatter('%d')
#minorLocator = mpl.ticker.MultipleLocator(1)
#minorFormatter = mpl.ticker.FormatStrFormatter('%d')

In [7]:
def draw_frequencie_ranges(ax):
    ax.axvline(x=140, color='red', linewidth=1)
    ax.axvline(x=144, color='red', linewidth=1)

def show_report(height, diameter):
    nec = sc_quad_helix(height, diameter)

    z = simulate_and_get_impedance(nec)

    print "Initial impedance: (%6.1f,%+6.1fI) Ohms" % (z.real, z.imag)
    print "VSWR @ 50 Ohm is %6.6f" % vswr(z, system_impedance)

    nec = sc_quad_helix(height, diameter)
  
    freqs, gains, vswrs = get_gain_swr_range(height, diameter, start=100, stop=200)

    freqs = np.array(freqs) / 1000000 # In MHz
    
    print len(freqs), len(gains), len(vswrs)
  
    ax = plt.subplot(111)
    ax.plot(freqs, gains)
    draw_frequencie_ranges(ax)

    ax.set_title("Gains of a 5-element log-periodic antenna")
    ax.set_xlabel("Frequency (MHz)")
    ax.set_ylabel("Gain")

    #ax.yaxis.set_major_locator(majorLocator)
    #ax.yaxis.set_major_formatter(majorFormatter)

    #ax.yaxis.set_minor_locator(minorLocator)
    #ax.yaxis.set_minor_formatter(minorFormatter)

    #ax.yaxis.grid(b=True, which='minor', color='0.75', linestyle='-')

    plt.show()

    ax = plt.subplot(111)
    ax.plot(freqs, vswrs)
    draw_frequencie_ranges(ax)

    ax.set_yscale("log")
    ax.set_title("VSWR of a QHA antenna @ 100 Ohm impedance")
    ax.set_xlabel("Frequency (MHz)")
    ax.set_ylabel("VSWR")

    #ax.yaxis.set_major_locator(majorLocator)
    #ax.yaxis.set_major_formatter(majorFormatter)
    #ax.yaxis.set_minor_locator(minorLocator)
    #ax.yaxis.set_minor_formatter(minorFormatter)
  
    #ax.yaxis.grid(b=True, which='minor', color='0.75', linestyle='-')
    plt.show()

In [11]:
  initial_height  = wavelength * 0.3
  initial_diameter  = wavelength * 0.2

  print "Wavelength is %0.4fm, initial height and diameter is %0.4fm, %0.4fm" % (wavelength, initial_height, initial_diameter)
  
  print "Unoptimized antenna..."
  show_report(initial_height, initial_diameter)

  print "Optimizing antenna..."
  target = create_optimization_target()

  # Optimize local minimum only with gradient desce
  #optimized_result = scipy.optimize.minimize(target, np.array([initial_height, initial_diameter]), method='Nelder-Mead')

  # Use differential evolution:
  minimizer_kwargs = dict(method='Nelder-Mead')
  bounds = [ (0.2, 0.9), (0.2, 0.9) ]
  optimized_result = scipy.optimize.differential_evolution(target, bounds, seed=42, disp=True, popsize=20)

  # Basin hopping isn't so good, but could also have been an option:
  #optimized_result = scipy.optimize.basinhopping(target, np.array([initial_height, initial_diameter]), minimizer_kwargs=minimizer_kwargs, niter=5, stepsize=0.015, T=2.0, disp=True)

  print "Optimized antenna..."
  optimized_height, optimized_diameter =  optimized_result.x[0], optimized_result.x[1]
  print "Wavelength is %0.4fm, optimized height and diameter is %0.4fm, %0.4fm" % (wavelength, optimized_height, optimized_diameter)
  show_report(optimized_height, optimized_diameter)



Wavelength is 2.0964m, initial height and diameter is 0.6289m, 0.4193m
Unoptimized antenna...
Initial impedance: (   8.9,-904.5I) Ohms
VSWR @ 50 Ohm is 1839.118421
101 101 101
Optimizing antenna...
(79.6095568516-1006.35998755j)
[ 259.32883786] 0.230309893982 0.884527708542
(7.51509176345-873.519441153j)
[ 2037.84736957] 0.481138402877 0.436499102672
(9.85412089967-881.132762495j)
[ 1581.94218094] 0.28786022865 0.516709261995
(15.6652095301-928.188083011j)
[ 1103.53850616] 0.682963249629 0.465484172165
(106.545841054-1035.46797723j)
[ 205.84424801] 0.708291477556 0.729748608056
(79.0654124568-1066.73565039j)
[ 293.1009712] 0.691548618786 0.866295781405
(2.68578546594-767.996462043j)
[ 4410.99077056] 0.514398560579 0.267658082551
(79.7862156926-1006.26222799j)
[ 258.57979703] 0.253516463213 0.860257625441
(13.6114251332-911.725154303j)
[ 1225.88140112] 0.520330740961 0.496605496902
(34.6720731526-942.496486025j)
[ 515.68272735] 0.473132034908 0.611601190371
(2.87809306294-796.187716322j

In [24]:
freqs, gains, vswrs = get_gain_swr_range(optimized_height, optimized_diameter, start=140, stop=145)

In [25]:
vswrs

[12390.649807184267,
 946.995356704651,
 135.5981107200613,
 4.592462432713049,
 54.920582416687466,
 170.81738832237357]

In [14]:
nec = sc_quad_helix(0.6, 0.4)
nec.set_frequency(143.05) # TODO: ensure that we don't need to re-generate this!
nec.radiation_pattern(thetas=Range(-180,180, count=90), phis=Range(90,90,count=1))

rp = nec.context.get_radiation_pattern(0)
ipt = nec.get_input_parameters(0)
z = ipt.get_impedance()[0]

In [15]:
  # Gains are in decibels
  gains_db = rp.get_gain()[:,0] # Is an array of theta,phi -> gain. In this case we only have one phi
  thetas = rp.get_theta_angles() * 3.1415 / 180.0
  phis = rp.get_phi_angles() * 3.1415 / 180.0

In [19]:
ax = plt.subplot(111, polar=True)

ax.plot(thetas, gains_db, color='r', linewidth=3)
ax.set_xticks(np.pi/180. * np.linspace(180,  -180, 8, endpoint=False))
ax.set_theta_zero_location("N")
ax.set_rlim((-24.0, 9.0)) # TODO: automate. TODO: 4nec2 cheats and makes the lowest points (-999) the same as the lowest non-999 point :)
ax.set_rticks(np.linspace(-24, 9, 10, endpoint=False))
ax.grid(True)

In [20]:
  ax.set_title("Gain pattern in the vertical plane", va='bottom')
  plt.show()


In [18]:
gains_db

array([-4.17740777, -4.16840921, -4.14139171, -4.09630422, -4.03310476,
       -3.95180978, -3.85253751, -3.73552921, -3.60113891, -3.44979227,
       -3.28192591, -3.09792722, -2.89809767, -2.68266005, -2.45182101,
       -2.20588695, -1.94541715, -1.67138554, -1.3853171 , -1.08936698,
       -0.78632187, -0.47951904, -0.17269569,  0.13020735,  0.42525793,
        0.70873423,  0.97729599,  1.22811457,  1.45895911,  1.66824154,
        1.85502637,  2.01901179,  2.16048816,  2.28027888,  2.37966743,
        2.46031362,  2.52416168,  2.57334313,  2.61007762,  2.63657531,
        2.65494478,  2.66710989,  2.6747385 ,  2.67918481,  2.68144574,
        2.68213101,  2.68144574,  2.67918481,  2.6747385 ,  2.66710989,
        2.65494478,  2.63657531,  2.61007762,  2.57334313,  2.52416168,
        2.46031362,  2.37966743,  2.28027888,  2.16048816,  2.01901179,
        1.85502637,  1.66824154,  1.45895911,  1.22811457,  0.97729599,
        0.70873423,  0.42525793,  0.13020735, -0.17269569, -0.47

In [9]:
%pylab qt
# This is the 3D plotting toolkit
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

x = gains_db*np.sin(phis)*np.cos(thetas)
y = gains_db*np.sin(phis)*np.sin(thetas)
z = gains_db*np.cos(thetas)

fig = plt.figure()
ax = plt.gca(projection='3d')
ax.set_aspect("equal")
ax.plot_surface(x, y, z)
plt.show()


Populating the interactive namespace from numpy and matplotlib


ValueError: operands could not be broadcast together with shapes (180,90) (180,) 

In [37]:
v

array([[ 0.        ,  0.34906585,  0.6981317 ,  1.04719755,  1.3962634 ,
         1.74532925,  2.0943951 ,  2.44346095,  2.7925268 ,  3.14159265],
       [ 0.        ,  0.34906585,  0.6981317 ,  1.04719755,  1.3962634 ,
         1.74532925,  2.0943951 ,  2.44346095,  2.7925268 ,  3.14159265],
       [ 0.        ,  0.34906585,  0.6981317 ,  1.04719755,  1.3962634 ,
         1.74532925,  2.0943951 ,  2.44346095,  2.7925268 ,  3.14159265],
       [ 0.        ,  0.34906585,  0.6981317 ,  1.04719755,  1.3962634 ,
         1.74532925,  2.0943951 ,  2.44346095,  2.7925268 ,  3.14159265],
       [ 0.        ,  0.34906585,  0.6981317 ,  1.04719755,  1.3962634 ,
         1.74532925,  2.0943951 ,  2.44346095,  2.7925268 ,  3.14159265],
       [ 0.        ,  0.34906585,  0.6981317 ,  1.04719755,  1.3962634 ,
         1.74532925,  2.0943951 ,  2.44346095,  2.7925268 ,  3.14159265],
       [ 0.        ,  0.34906585,  0.6981317 ,  1.04719755,  1.3962634 ,
         1.74532925,  2.0943951 ,  2.44346095

In [2]:
def add_ground_screen(geo, start_tag, length):
    # The ground is modeled 6 radial wires (brass, radius 1 mm), equally spaced by 2pi/6, and their initial length is d = 2 cm

    src = np.array([0, 0, 0])
    radius = 0.001
    
    for i in range(0, 6):
        angle = i * (2*np.pi/6.0)
        dst = np.array([length * np.cos(angle), length * np.sin(angle), 0])
        # TODO: nr_segments (actually, more TODO: if nr_segments>=9, the geometry is invalid with this wire radius!)
        geo.wire(tag_id=start_tag+i, nr_segments=5, src=src, dst=dst, radius=radius) # TODO: nr_segments


In [3]:
def geometry_monopole_ground(freq_mhz, monopole_length, ground_wire_length, nr_segments):
  wire_radius = 0.0005 # 0.5 mm

  nec = context_clean(nec_context())
  nec.set_extended_thin_wire_kernel(True)

  geo = geometry_clean(nec.get_geometry())
  
  bottom = np.array([0, 0, 0])
  top    = np.array([0, 0, monopole_length])

  wire_tag = 1
  geo.wire(tag_id=wire_tag, nr_segments=nr_segments, src=bottom, dst=top, radius=wire_radius)

  add_ground_screen(geo, start_tag=2, length=ground_wire_length)

  # Everything is in brass
  nec.set_wire_conductivity(brass_conductivity)

  nec.geometry_complete(ground_plane=False) # We added our own 'ground plane'

  nec.voltage_excitation(wire_tag=wire_tag, segment_nr=1, voltage=1.0)

  return nec

In [8]:
geo_opt = geometry_monopole_ground(143, 0.5, 0.5, 10)
geo_opt.set_frequency(143)
geo_opt.radiation_pattern(thetas=Range(-90, 90, count=90), phis=Range(90,90,count=1))

In [9]:
  #get the radiation_pattern
  rp = geo_opt.context.get_radiation_pattern(0)

In [10]:
  # Gains are in decibels
  gains_db = rp.get_gain() # Is an array of theta,phi -> gain. In this case we only have one phi
  thetas = rp.get_theta_angles() * 3.1415 / 180.0
  phis = rp.get_phi_angles() * 3.1415 / 180.0

In [11]:
ax = plt.subplot(111, polar=True)
ax.plot(thetas, gains_db, color='r', linewidth=3)
ax.set_xticks(np.pi/180. * np.linspace(180,  -180, 8, endpoint=False))
ax.set_theta_zero_location("N")
ax.set_rlim((-24.0, 9.0)) # TODO: automate. TODO: 4nec2 cheats and makes the lowest points (-999) the same as the lowest non-999 point :)
ax.set_rticks(np.linspace(-24, 9, 10, endpoint=False))
ax.grid(True)

ax.set_title("Gain pattern in the vertical plane", va='bottom')
plt.show()

In [34]:
# -*- coding: utf-8 -*-
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
#from itertools import product, combinations

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_aspect("equal")


thetas=np.arange(np.pi, 0, np.pi/10)
phis=np.arange(np.pi/2, 0,np.pi/90)


x=np.array(0)
y=np.array(0)
z=np.array(0)
phi_gain = 0

for phi in phis:
    theta_gain =0
    for theta in thetas:
#        x = np.append(x, gains_db[theta_gain,phi_gain]*np.sin(phi)*np.cos(theta))
#        y = np.append(y, gains_db[theta_gain,phi_gain]*np.sin(phi)*np.sin(theta))
#        z = np.append(z, gains_db[theta_gain,phi_gain]*np.cos(theta))
        x = np.append(x, 1*np.sin(phi)*np.cos(theta))
        y = np.append(y, 1*np.sin(phi)*np.sin(theta))
        z = np.append(z, 1*np.cos(theta))        
        
        theta_gain =+ 1
    phi_gain =+1
        
ax.plot_surface(x, y, z, color="r")


plt.show()

In [15]:
rp.get_phi_angles()

array([ 90.,  88.,  86.,  84.,  82.,  80.,  78.,  76.,  74.,  72.,  70.,
        68.,  66.,  64.,  62.,  60.,  58.,  56.,  54.,  52.,  50.,  48.,
        46.,  44.,  42.,  40.,  38.,  36.,  34.,  32.,  30.,  28.,  26.,
        24.,  22.,  20.,  18.,  16.,  14.,  12.,  10.,   8.,   6.,   4.,
         2.])

In [20]:
Range(3.14/2,0,count=45)[0]

TypeError: 'Range' object does not support indexing

In [36]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.mlab as mlab
import numpy as np

def P(X, Y):
    return mlab.bivariate_normal(X, Y, 1.0, 1.0, 0.0, 0.0)      

fig = plt.figure()
ax = fig.gca(projection = '3d')
jet = plt.get_cmap('jet')

x = np.linspace(-2, 2, 60)
y = np.linspace(-2, 2, 60)
X, Y = np.meshgrid(x, y)
Z = P(X, Y)
surf = ax.plot_surface(X, Y, Z, rstride = 1, cstride = 1, cmap = jet, linewidth = 0)
ax.set_zlim3d(0, Z.max())

plt.show()

In [37]:
np.meshgrid

<function numpy.lib.function_base.meshgrid>

In [2]:
import numpy
from mayavi2.tools import imv
x=numpy.arange(-8,8,.2)
def f(x,y):
    r=numpy.sqrt(x**2+y**2)+.01
    return numpy.sin(r)/r
imv.surf(x,x,f)

ImportError: No module named mayavi2.tools

In [ ]:
% pylab qt
import numpy
from mayavi.mlab import *

def test_imshow():
    """ Use imshow to visualize a 2D 10x10 random array.
    """
    s = numpy.random.random((10, 10))
    return imshow(s, colormap='gist_earth')